<a href="https://colab.research.google.com/github/phu-minh/CS116.M11.KHTN/blob/main/6_Classifiers_sklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/CS116"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/CS116


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from statistics import mean
from sklearn.decomposition import PCA

# Social Network Ads - Classification

In [3]:
df1 = pd.read_csv('Social_Network_Ads.csv')
df1.head()

,Age,EstimatedSalary,Purchased
0,19,19000,0
1,35,20000,0
2,26,43000,0
3,27,57000,0
4,19,76000,0


In [4]:
x1 = np.array(df1.drop('Purchased', axis=1))
y1 = np.array(df1['Purchased'])

In [5]:
print(x1[[1,2,3,4,5]])

[[   35 20000]
 [   26 43000]
 [   27 57000]
 [   19 76000]
 [   27 58000]]


In [6]:
print(y1[:5])

[0 0 0 0 0]


In [7]:
def pipeline(x, y, model):
    accuracies = []
    kf = KFold(n_splits=5)
    for train_index, test_index in kf.split(x):
        x_train, y_train = x[train_index], y[train_index]
        x_test, y_test = x[test_index], y[test_index]
        model.fit(x_train, y_train)

        score = model.score(x_test, y_test)
        accuracies.append(score)

    return accuracies

def pipeline_pca(x, y, model, pca, test_pca_flag=False):
    '''
    test_pca_flag: bool
        True: apply PCA for BOTH train_set and test_set
        False:  fit PCA on train then apply on test immediately
    '''
    accuracies = []
    kf = KFold(n_splits=5)
    for train_index, test_index in kf.split(x):
        x_train, y_train = x[train_index], y[train_index]
        x_test, y_test = x[test_index], y[test_index]

        pca.fit(x_train)
        transformed_x_train = pca.transform(x_train)

        if (test_pca_flag):
            pca.fit(x_test)
        transformed_x_test = pca.transform(x_test)

        model.fit(transformed_x_train, y_train)

        score = model.score(transformed_x_test, y_test)
        accuracies.append(score)

    return accuracies

In [8]:
results1 = {}
results1['logistic_regression'] = {'model': LogisticRegression(solver='liblinear', random_state=0)}
results1['svm'] = {'model': make_pipeline(StandardScaler(), SVC(gamma='auto'))}
results1['naive_gaussian'] = {'model': GaussianNB()}
results1['decision_tree'] = {'model': DecisionTreeClassifier()}
results1['random_forest'] = {'model': RandomForestClassifier()}
results1['knn'] = {'model': KNeighborsClassifier(n_neighbors=1)} 

In [10]:
for method in results1:
    accuracies = pipeline(x1, y1, results1[method]['model'])
    results1[method]['accuracy'] = mean(accuracies)
    print(f'Accuracy of {method}: {mean(accuracies)}')

Accuracy of logistic_regression: 0.6224999999999999
Accuracy of svm: 0.9099999999999999
Accuracy of naive_gaussian: 0.855
Accuracy of decision_tree: 0.8525
Accuracy of random_forest: 0.89
Accuracy of knn: 0.7625


In [11]:
res_df1 = pd.DataFrame(results1).T.drop(columns=['model'])
res_df1

,accuracy
logistic_regression,0.6225
svm,0.91
naive_gaussian,0.855
decision_tree,0.8525
random_forest,0.89
knn,0.7625


# Wine - Classification with PCA

In [12]:
# show data structure
df2 = pd.read_csv('Wine.csv')
df2.head()

,Alcohol,Malic_Acid,Ash,Ash_Alcanity,Magnesium,Total_Phenols,Flavanoids,Nonflavanoid_Phenols,Proanthocyanins,Color_Intensity,Hue,OD280,Proline,Customer_Segment
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065,1
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050,1
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185,1
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480,1
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735,1


In [13]:
x2 = np.array(df2.drop('Customer_Segment', axis=1))
y2 = np.array(df2['Customer_Segment'])

In [14]:
print(x2[:5])

[[1.423e+01 1.710e+00 2.430e+00 1.560e+01 1.270e+02 2.800e+00 3.060e+00
  2.800e-01 2.290e+00 5.640e+00 1.040e+00 3.920e+00 1.065e+03]
 [1.320e+01 1.780e+00 2.140e+00 1.120e+01 1.000e+02 2.650e+00 2.760e+00
  2.600e-01 1.280e+00 4.380e+00 1.050e+00 3.400e+00 1.050e+03]
 [1.316e+01 2.360e+00 2.670e+00 1.860e+01 1.010e+02 2.800e+00 3.240e+00
  3.000e-01 2.810e+00 5.680e+00 1.030e+00 3.170e+00 1.185e+03]
 [1.437e+01 1.950e+00 2.500e+00 1.680e+01 1.130e+02 3.850e+00 3.490e+00
  2.400e-01 2.180e+00 7.800e+00 8.600e-01 3.450e+00 1.480e+03]
 [1.324e+01 2.590e+00 2.870e+00 2.100e+01 1.180e+02 2.800e+00 2.690e+00
  3.900e-01 1.820e+00 4.320e+00 1.040e+00 2.930e+00 7.350e+02]]


In [15]:
print(y2[:5])

[1 1 1 1 1]


In [16]:
results2 = {}
results2['logistic_regression'] = {'model': LogisticRegression(solver='liblinear', random_state=0)}
results2['svm'] = {'model': make_pipeline(StandardScaler(), SVC(gamma='auto'))}
results2['naive_gaussian'] = {'model': GaussianNB()}
results2['decision_tree'] = {'model': DecisionTreeClassifier()}
results2['random_forest'] = {'model': RandomForestClassifier()}
results2['knn'] = {'model': KNeighborsClassifier(n_neighbors=1)} 

In [18]:
for method in results2:
    accuracies = pipeline(x2, y2, results2[method]['model'])
    results2[method]['normal_accuracy'] = mean(accuracies)
    print(f'Accuracy of {method}: {mean(accuracies)}')

    accuracies = pipeline_pca(x2, y2, results2[method]['model'], pca=PCA(n_components=3))
    results2[method]['pca_accuracy'] = mean(accuracies)
    print(f'Accuracy of {method} with pca: {mean(accuracies)}')
   
    accuracies = pipeline_pca(x2, y2, results2[method]['model'], pca=PCA(n_components=3), test_pca_flag=True)
    results2[method]['test_pca_accuracy'] = mean(accuracies)
    print(f'Accuracy of {method} with pca and test_pca_flag: {mean(accuracies)}\n')
    

Accuracy of logistic_regression: 0.933015873015873
Accuracy of logistic_regression with pca: 0.6206349206349207
Accuracy of logistic_regression with pca and test_pca_flag: 0.15761904761904763

Accuracy of svm: 0.954920634920635
Accuracy of svm with pca: 0.6152380952380953
Accuracy of svm with pca and test_pca_flag: 0.14063492063492064

Accuracy of naive_gaussian: 0.9326984126984127
Accuracy of naive_gaussian with pca: 0.6546031746031746
Accuracy of naive_gaussian with pca and test_pca_flag: 0.1238095238095238

Accuracy of decision_tree: 0.8257142857142857
Accuracy of decision_tree with pca: 0.6719047619047619
Accuracy of decision_tree with pca and test_pca_flag: 0.2195238095238095

Accuracy of random_forest: 0.9665079365079365
Accuracy of random_forest with pca: 0.6552380952380953
Accuracy of random_forest with pca and test_pca_flag: 0.14047619047619048

Accuracy of knn: 0.6333333333333333
Accuracy of knn with pca: 0.638888888888889
Accuracy of knn with pca and test_pca_flag: 0.1807936

In [19]:
df_res2 = pd.DataFrame(results2).T
df_res2.head()

,model,normal_accuracy,pca_accuracy,test_pca_accuracy
logistic_regression,"LogisticRegression(C=1.0, class_weight=None, d...",0.933016,0.620635,0.157619
svm,"(StandardScaler(copy=True, with_mean=True, wit...",0.954921,0.615238,0.140635
naive_gaussian,"GaussianNB(priors=None, var_smoothing=1e-09)",0.932698,0.654603,0.12381
decision_tree,"DecisionTreeClassifier(ccp_alpha=0.0, class_we...",0.825714,0.671905,0.219524
random_forest,"(DecisionTreeClassifier(ccp_alpha=0.0, class_w...",0.966508,0.655238,0.140476
